# From JSON to Time Travel in 10 Seconds

This notebook demonstrates the power of Apache Iceberg in just a few lines of code:

* Read JSON data and create an Iceberg table
* Query the table instantly with SQL
* Append more data and query again
* Travel back in time to the first snapshot

All of this happens in seconds, with full ACID transactions and automatic versioning.

In [ ]:
import daft
import pyarrow as pa
import json
from pathlib import Path
from pyiceberg.catalog.sql import SqlCatalog

# Setup paths and create Iceberg catalog
warehouse_path = Path('../data/warehouse_quick_demo').absolute()
warehouse_path.mkdir(parents=True, exist_ok=True)
catalog_db = warehouse_path / 'catalog.db'
catalog_db.unlink(missing_ok=True)  # Fresh start

catalog = SqlCatalog(
    'demo',
    **{'uri': f'sqlite:///{catalog_db}', 'warehouse': f'file://{warehouse_path}'}
)
catalog.create_namespace('demo')
print("✅ Catalog ready")

In [ ]:
# Read first 100K events from JSON and create Iceberg table
jsonl_file = Path('../data/input/events.jsonl')

with jsonl_file.open('r') as f:
    # Read first 100K records
    data = [json.loads(line) for i, line in enumerate(f) if i < 100000]

arrow_table = pa.Table.from_pylist(data)
iceberg_table = catalog.create_table('demo.events', schema=pa.schema(arrow_table.schema))
iceberg_table.append(arrow_table)

print(f"✅ Created table with {len(data):,} events (Snapshot 1)")

# Query with Daft
df = daft.read_iceberg(iceberg_table)
print("\nTop 5 event types:")
daft.sql("""
    SELECT type, COUNT(*) as count
    FROM df
    GROUP BY type
    ORDER BY count DESC
    LIMIT 5
""").show()

In [ ]:
# Append next 100K events
with jsonl_file.open('r') as f:
    # Skip first 100K, read next 100K
    data = [json.loads(line) for i, line in enumerate(f) if 100000 <= i < 200000]

arrow_table = pa.Table.from_pylist(data)
iceberg_table.append(arrow_table)

print(f"✅ Appended {len(data):,} more events (Snapshot 2)")

# Query again - now includes both batches
df = daft.read_iceberg(iceberg_table)
print("\nTotal events after append:")
daft.sql("SELECT COUNT(*) as total FROM df").show()

In [ ]:
# Time travel: go back to first snapshot
history = iceberg_table.history()
first_snapshot_id = history[0].snapshot_id

print(f"⏰ Time traveling to Snapshot {first_snapshot_id}\n")

# Read data as it was in the first snapshot
df_past = daft.read_iceberg(iceberg_table, snapshot_id=first_snapshot_id)
print("Events in first snapshot:")
daft.sql("SELECT COUNT(*) as total FROM df_past").show()

print("\n🎉 You just:")
print("  • Created a versioned table from JSON")
print("  • Queried it with SQL")
print("  • Appended new data")
print("  • Traveled back in time")
print("\nAll with ACID transactions, no data rewrites, in seconds!")